# Crawling Tweet

In [ ]:
pip install playwright pandas



     -------------------------------------- 35.6/35.6 MB 692.5 kB/s eta 0:00:00
     ------------------------------------ 298.7/298.7 kB 499.0 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [34]:
import asyncio
from playwright.async_api import async_playwright
import pandas as pd
import time

keywords = ["#bitcoin"]
max_tweets_per_keyword = 50

all_tweets = []

async def scrape_tweets(keyword, max_tweets):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        search_url = f"https://twitter.com/search?q={keyword}&f=live"
        await page.goto(search_url)
        await asyncio.sleep(3)  # tunggu halaman load

        tweets = set()
        last_height = 0
        while len(tweets) < max_tweets:
            page_content = await page.query_selector_all("article div[lang]")
            for el in page_content:
                text = await el.inner_text()
                if text not in tweets:
                    tweets.add(text)
                    all_tweets.append({
                        "keyword": keyword,
                        "content": text
                    })
                    if len(tweets) >= max_tweets:
                        break

            # scroll ke bawah
            await page.evaluate("window.scrollBy(0, document.body.scrollHeight)")
            await asyncio.sleep(2)

            new_height = await page.evaluate("document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        await browser.close()
        print(f"✅ Selesai ambil {len(tweets)} tweet untuk '{keyword}'")

async def main():
    for kw in keywords:
        await scrape_tweets(kw, max_tweets_per_keyword)

# Jalankan asyncio loop
await main()

# Simpan ke CSV
df = pd.DataFrame(all_tweets)
df.to_csv("tweets_playwright_async.csv", index=False, encoding="utf-8-sig")
print("✅ Semua tweet tersimpan di tweets_playwright_async.csv")


NotImplementedError: 